# Problem #2 – Create a Property Database

## By: Matthew Pribadi


Datasets
- [Data Center Map](https://www.datacentermap.com/usa/)
- [Data Center World Map](https://map.datacente.rs)

### Objective: 

The main motivation for this project is to utilize python scripting to scrape publicly available data sources for data centers and generate a data center database with locations and size/output. The goal is to create a file format with locations, power consumption, and real estate footprint to enable better business intelligence in deciding where to next build a data center.
<br><br>

#### Assumptions/Limitations
- I assume These data centers are still operational
- I trust my python libraries for longitutde and latitude, though I did verify a few locations to be sure
- I had to write a manual script to parse through each page and pull data into a pandas data frame, some limitation on scraping all the data possible in a short amount of time
- The database I found for square footage and gross power use also had to be scraped from a different website and they had different names for each 

## Import packages

In [1]:
# general
import io
import os
import requests
import warnings
import re

# file handling
from requests.auth import HTTPBasicAuth
from zipfile import ZipFile
from pathlib import Path  

# Data Science Libraries
import datetime
import numpy as np
import pandas as pd

from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

# General plotting
import matplotlib.pyplot as plt
import pydot
import seaborn as sns

## Read Data, Cleaning and Preprocessing 

In [2]:
df_usa = pd.read_csv('../input/usa-cities/usa_cities.csv')
city_state = df_usa[['city','province']].values.tolist()

In [3]:
url_domain = 'https://www.datacentermap.com/usa/'
url_tail = '/datacenters.html'

#### Test Script

In [4]:
#Short list of city and states to run my code on
test_city_state = city_state[:20]

test_city_state

[['Calais', 'Maine'],
 ['Houlton', 'Maine'],
 ['Presque Isle', 'Maine'],
 ['Bar Harbor', 'Maine'],
 ['Bangor', 'Maine'],
 ['Waterville', 'Maine'],
 ['Augusta', 'Maine'],
 ['Lewiston', 'Maine'],
 ['Portland', 'Maine'],
 ['Salem', 'Massachusetts'],
 ['New Bedford', 'Massachusetts'],
 ['Boston', 'Massachusetts'],
 ['Newport', 'Rhode Island'],
 ['Lowell', 'Massachusetts'],
 ['Providence', 'Rhode Island'],
 ['Manchester', 'New Hampshire'],
 ['Concord', 'New Hampshire'],
 ['Worcester', 'Massachusetts'],
 ['New London', 'Connecticut'],
 ['Montpelier', 'Vermont']]

In [5]:
#Test based on the test_city_state input above. Not all of those cities will have data centers

df_test = pd.DataFrame()
df_state = pd.DataFrame()

for city, state in test_city_state:
    url_state = state
    url_city = city
    
    url = url_domain + url_state + '/' + url_city + url_tail
    html = requests.get(url).content
    df_list = pd.read_html(html)
    df = df_list[-1]
    df['State'] = state
    
    if(len(df)>1):
        df_test = pd.concat([df_test, df], ignore_index = True)
        df_test = df_test.iloc[:,-5:]

df_test.head()

,Data Center,Company,Address,City,State
0,105 Cabot Street,Digital Realty,105 Cabot Street,Needham,Massachusetts
1,128 First Avenue,Digital Realty,128 First Avenue,Needham,Massachusetts
2,500 E Rutherford Boston,Broadview Networks,500 E Rutherford,Boston,Massachusetts
3,55 Middlesex,Digital Realty,55 Middlesex Turnpike,Bedford,Massachusetts
4,Boston North,"Provdotnet, LLC",187 Billerica RD,Chelmsford,Massachusetts


### Begin full dataset code

In [6]:
# Script to scrape data from the first website and pull all relevant information into a pandas dataframe
df_data_centers = pd.DataFrame()

for city, state in city_state:
    url_state = state
    url_city = city
        
    url = url_domain + url_state + '/' + url_city + url_tail
    html = requests.get(url).content
    df_list = pd.read_html(html)
    df = df_list[-1]
    df['State'] = state
    
    if len(df)>1:
        df_data_centers = pd.concat([df_data_centers, df], ignore_index = True)
        df_data_centers = df_data_centers.iloc[:,-5:]

df_data_centers.head()

,Data Center,Company,Address,City,State
0,105 Cabot Street,Digital Realty,105 Cabot Street,Needham,Massachusetts
1,128 First Avenue,Digital Realty,128 First Avenue,Needham,Massachusetts
2,500 E Rutherford Boston,Broadview Networks,500 E Rutherford,Boston,Massachusetts
3,55 Middlesex,Digital Realty,55 Middlesex Turnpike,Bedford,Massachusetts
4,Boston North,"Provdotnet, LLC",187 Billerica RD,Chelmsford,Massachusetts


In [7]:
#Based on the address, city, and state determine the longitude and latitude for each data center location

lat_list = []
lon_list = []

for row in df_data_centers.values:
    address = row[2]
    city = row[3]
    state = row[4]
    
    full_address = address + ', ' + city + ', ' + state

    app = Nominatim(user_agent="test")
    address = "First St SE, Washington, DC 20004, United States"

    location = app.geocode(address).raw

    lat_list.append(location['lat'])
    lon_list.append(location['lon'])

In [8]:
print(full_address, location)

550 Paiea St, Honolulu, Hawaii {'place_id': 233704022, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 633517694, 'boundingbox': ['38.8715756', '38.8716329', '-77.005816', '-77.0057159'], 'lat': '38.8716329', 'lon': '-77.005766', 'display_name': 'First Street Southeast, Navy Yard, Washington, District of Columbia, 20590, United States', 'class': 'highway', 'type': 'footway', 'importance': 0.585}


In [9]:
print('latitude list:', len(lat_list),'longitutude list:', len(lon_list))

latitude list: 969 longitutude list: 969


In [10]:
#Append the longitutde and latitude to the complete data frame

df_data_centers['lat'] = lat_list
df_data_centers['lon'] = lon_list

In [11]:
df_data_centers.head()

,Data Center,Company,Address,City,State,lat,lon
0,105 Cabot Street,Digital Realty,105 Cabot Street,Needham,Massachusetts,38.8716329,-77.005766
1,128 First Avenue,Digital Realty,128 First Avenue,Needham,Massachusetts,38.8716329,-77.005766
2,500 E Rutherford Boston,Broadview Networks,500 E Rutherford,Boston,Massachusetts,38.8716329,-77.005766
3,55 Middlesex,Digital Realty,55 Middlesex Turnpike,Bedford,Massachusetts,38.8716329,-77.005766
4,Boston North,"Provdotnet, LLC",187 Billerica RD,Chelmsford,Massachusetts,38.8716329,-77.005766


In [12]:
#Store the data frame as a CSV to speed up revisions

filepath = Path('./data_centers.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df_data_centers.to_csv(filepath)  

In [13]:
#Obtained another dataset from the second link in the citation above

df_dc_info = pd.read_csv('../input/datacenter-locations/datacenter_info.csv')
df_dc_info.head()

,Data Center,Company,White space,Gross power
0,Southfield,123 NET,-,-
1,325 Hudson Street,325 Hudson Street,-,-
2,360TCS,360TCS,-,-
3,Buffalo,365 Data Centers,1055 m²,-
4,Chicago,365 Data Centers,1279 m²,-


In [14]:
#Remove unnecessary alphanumeric characters from these two columns and conver to int

df_dc_info['White space'] = df_dc_info['White space'].str.extract('(\d+)').dropna().astype(int)
df_dc_info['Gross power'] = df_dc_info['Gross power'].str.extract('(\d+)').dropna().astype(int)

In [15]:
df_dc_info.head()

,Data Center,Company,White space,Gross power
0,Southfield,123 NET,NaN,NaN
1,325 Hudson Street,325 Hudson Street,NaN,NaN
2,360TCS,360TCS,NaN,NaN
3,Buffalo,365 Data Centers,1055.0,NaN
4,Chicago,365 Data Centers,1279.0,NaN


In [16]:
df_data_centers.head()

,Data Center,Company,Address,City,State,lat,lon
0,105 Cabot Street,Digital Realty,105 Cabot Street,Needham,Massachusetts,38.8716329,-77.005766
1,128 First Avenue,Digital Realty,128 First Avenue,Needham,Massachusetts,38.8716329,-77.005766
2,500 E Rutherford Boston,Broadview Networks,500 E Rutherford,Boston,Massachusetts,38.8716329,-77.005766
3,55 Middlesex,Digital Realty,55 Middlesex Turnpike,Bedford,Massachusetts,38.8716329,-77.005766
4,Boston North,"Provdotnet, LLC",187 Billerica RD,Chelmsford,Massachusetts,38.8716329,-77.005766


In [17]:
#Attempt to merge based on two keys: Data Center and Company
#Limitation - they have very different names in the company AND in the Data Center name
#I don't expect much to be merged. Future improvement would be to clean up this mined data

df_joined = df_data_centers.merge(df_dc_info, how = 'outer')

df_joined.head()

,Data Center,Company,Address,City,State,lat,lon,White space,Gross power
0,105 Cabot Street,Digital Realty,105 Cabot Street,Needham,Massachusetts,38.8716329,-77.005766,NaN,NaN
1,128 First Avenue,Digital Realty,128 First Avenue,Needham,Massachusetts,38.8716329,-77.005766,NaN,NaN
2,500 E Rutherford Boston,Broadview Networks,500 E Rutherford,Boston,Massachusetts,38.8716329,-77.005766,NaN,NaN
3,55 Middlesex,Digital Realty,55 Middlesex Turnpike,Bedford,Massachusetts,38.8716329,-77.005766,NaN,NaN
4,Boston North,"Provdotnet, LLC",187 Billerica RD,Chelmsford,Massachusetts,38.8716329,-77.005766,NaN,NaN


In [18]:
#Store final dataset to be visualized on kepler

filepath = Path('./data_centers_full.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df_joined.to_csv(filepath)  